## YOLO

### PyTorch 기반 물체인식 모델
- CNN, rCNN(Regions with CNN)
- https://github.com/ultralytics/ultralytics 참조

#### YOLOv5 이상 설치
```shell
> pip install ultralytics
```

In [1]:
!pip install ultralytics

   ---------------------------------------- 0.0/974.8 kB ? eta -:--:--
   ----- ---------------------------------- 143.4/974.8 kB 4.3 MB/s eta 0:00:01
   --------------- ------------------------ 368.6/974.8 kB 4.6 MB/s eta 0:00:01
   ---------------------------------- ----- 829.4/974.8 kB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 974.8/974.8 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.6/39.5 MB 18.8 MB/s eta 0:00:03
   - -------------------------------------- 1.5/39.5 MB 15.6 MB/s eta 0:00:03
   -- ------------------------------------- 2.6/39.5 MB 20.6 MB/s eta 0:00:02
   --- ------------------------------------ 3.7/39.5 MB 21.7 MB/s eta 0:00:02
   ----- ---------------------------------- 5.2/39.5 MB 22.2 MB/s eta 0:00:02
   ------ --------------------------------- 6.9/39.5 MB 24.5 MB/s eta 0:00:02
   -------- ------------------------------- 8.4/39.5 MB 26.8 MB/s eta 0


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 콘솔에서 예측

In [3]:
# 콘솔에서 예측
## yolo11n.pt - pretrained model
!yolo predict model=yolo11n.pt source='./bus.jpg'

Ultralytics 8.3.109 🚀 Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 c:\Source\iot-dataanalysis-2025\day08\bus.jpg: 640x480 4 persons, 1 bus, 32.0ms
Speed: 2.1ms preprocess, 32.0ms inference, 78.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\Source\iot-dataanalysis-2025\runs\detect\predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


####  파이썬으로 예측

In [4]:
from ultralytics import YOLO


In [5]:
model = YOLO("./yolo11n.pt")

##### coco8.yaml
- https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco8.yaml
- https://github.com/ultralytics/assets/releases/download/v0.0.0/coco8.zip
- 위 내용대로 훈련을 시킨 결과 -> yolo11n.pt

In [ ]:
# coco8.yaml - YOLO 훈련에 사용할 데이터셋 정의파일
train_results = model.train(
    data='./coco8.yaml',
    epochs = 100,
    imgsz = 640,
    device = 'cuda:0'
)

#### 이미지예측

In [10]:
result = model('./0000001.jpg')


image 1/1 c:\Source\iot-dataanalysis-2025\day08\0000001.jpg: 480x640 1 cat, 106.0ms
Speed: 3.3ms preprocess, 106.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


In [11]:
# matplotlib 모듈 로드
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [15]:
img = result[0].plot()
img_pil = Image.fromarray(img[...,::-1])
img_pil.save('./predict_result.jpg')

### OpenCV
- Opensource Computer Vision 약자. 실시간 컴퓨터 비전(시각처리)을 목적으로 프로그래밍 라이브러리
- 인텔에서 2000년에 C, C++ 사용하기 위해서 개발
- 파이썬에서 사용할 수 있게 래핑

In [16]:
!pip install opencv-python


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import cv2
cv2.__version__

'4.11.0'

In [18]:
img2 = cv2.imread('./predict_result.jpg')
img2.shape

(464, 640, 3)

In [22]:
cv2.imshow('result', img2)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>

##### YOLO 예측

In [25]:
img = cv2.imread('./0000002.jpg')
resized_img = cv2.resize(img, (640,400))

result = model(resized_img)
plots = result[0].plot()

cv2.imshow('predict_result', plots)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 416x640 1 cup, 1 bowl, 1 dining table, 1 tv, 1 mouse, 1 book, 189.6ms
Speed: 12.0ms preprocess, 189.6ms inference, 3.1ms postprocess per image at shape (1, 3, 416, 640)


#### YOLO 실시간 예측

In [33]:
video_path = './sample01.mp4'
output_path = './sample01_output.mp4'
count_path = './sample01_count.mp4'

In [29]:
# 동영상 플레이
cap = cv2.VideoCapture(video_path)  # 0 -> 웹캠이나 카메라 설치된 번호

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    cv2.imshow('Video play', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import time

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0/fps
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width,height))

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret: break

    results = model(frame)
    for result in results:
        detect_frame = result.plot()
    out.write(detect_frame)
    cv2.imshow('YOLO Object Detection', detect_frame)
    cv2.imshow('Video play', frame)

    elapsed_time = time.time() - start_time
    delay = max(int((frame_time - elapsed_time) * 1000), 1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
!pip install shapely

In [ ]:
!pip install lap

In [ ]:
import cv2
from ultralytics.solutions import ObjectCounter

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), 'Error reading video file'

region_points = [(20, 400), (1080, 400)]
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(count_path, fourcc, fps, (width,height))

# 물체 카운터 초기화
counter = ObjectCounter(
    show=True,
    region=region_points,
    model='yolo11n.pt'
)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    results = counter(frame)
    out.write(results.plot_im)
    # cv2.imshow('YOLO Counting', frame)

cap.release()
out.release()
cv2.destroyAllWindows()